# Work with Text and Images

You need to run in the terminal:
```bash
pip install matplotlib
```

```bash
pip install -U torch torchvision
pip install -U "transformers>=4.41.0"
pip install -U qwen-vl-utils
pip install -U pillow
```


# Setup

In [ ]:
# add auto reload for notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import io
import shutil
import base64

from conversational_toolkit.chunking.pdf_chunker import PDFChunker

from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.embeddings.clip import CLIPEmbeddings
from conversational_toolkit.embeddings.qwen_vl import Qwen3VLEmbeddings
from conversational_toolkit.vectorstores.chromadb import ChromaDBVectorStore
from conversational_toolkit.retriever.vectorstore_retriever import (
    CompositeVectorStoreRetriever,
)
from conversational_toolkit.llms.base import LLMMessage, MessageContent, Roles
from conversational_toolkit.llms.openai import OpenAILLM

import matplotlib.pyplot as plt

# Parse document and chunk

In [ ]:
chunker = PDFChunker()

In [ ]:
chunks = chunker.make_chunks(
    "../../data/EPD_cardboard_redbox_cartonpallet.pdf",
    write_images=True,
    image_path="../../data/tmp_images/",
)

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk.title} ({chunk.mime_type})")
    if chunk.mime_type.startswith("image/"):
        image_data = base64.b64decode(chunk.content)

        image = plt.imread(io.BytesIO(image_data), format=chunk.mime_type.split("/")[1])

        plt.figure(figsize=(2, 2))
        plt.imshow(image)
        plt.axis("off")

plt.show()

# Embed Text

In [ ]:
text_embedding_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# if os.path.exists("../../db/db_text_tmp/"):
#    shutil.rmtree("../../db/db_text_tmp/")
text_vector_store = ChromaDBVectorStore(db_path="../../db/db_text_tmp/")

text_chunks = [c for c in chunks if c.mime_type.startswith("text/")]

embeddings = await text_embedding_model.get_embeddings([c.content for c in text_chunks])

await text_vector_store.insert_chunks(chunks=text_chunks, embedding=embeddings)

print(sum(len(e) for e in embeddings))

In [ ]:
query = "What is the amount of PBT/vPvB substances ?"

query_embedding = await text_embedding_model.get_embeddings(query)

results = await text_vector_store.get_chunks_by_embedding(query_embedding, top_k=5)

print(len(results))

for i, result in enumerate(results):
    print(f"Result {i}: {result.title} ({result.mime_type})")

print(results[0].content)

# Embed Images

In [ ]:
# image_embedding_model = CLIPEmbeddings()
image_embedding_model = Qwen3VLEmbeddings()

# if os.path.exists("../../db/db_image_siglip_tmp/"):
#     shutil.rmtree("../../db/db_image_siglip_tmp/")

image_vector_store = ChromaDBVectorStore(db_path="../../db/db_image_siglip_tmp/")

image_chunks = [c for c in chunks if c.mime_type.startswith("image/")]

embeddings = await image_embedding_model.get_embeddings([c for c in image_chunks])

await image_vector_store.insert_chunks(chunks=image_chunks, embedding=embeddings)

print(sum(len(e) for e in embeddings))

In [ ]:
query = "A carboard box on a pallet"

query_embedding = await image_embedding_model.get_text_embeddings(query)

results = await image_vector_store.get_chunks_by_embedding(query_embedding, top_k=5)

for i, result in enumerate(results):
    print(f"Result {i}: {result.title} ({result.mime_type})")
    image_data = base64.b64decode(result.content)

    image = plt.imread(io.BytesIO(image_data), format=result.mime_type.split("/")[1])

    plt.figure(figsize=(2, 2))
    plt.imshow(image)
    plt.axis("off")

# Create Composite Vector Store

In [ ]:
query = "What is the amount of PBT/vPvB substances ?"

composite_retriever = CompositeVectorStoreRetriever(
    embedding_models=[text_embedding_model, image_embedding_model],
    vector_stores=[text_vector_store, image_vector_store],
    top_k=[3, 1],
)

results = await composite_retriever.retrieve(query)

for i, result in enumerate(results):
    # if text print it
    print(f"Result {i} - type: {result.mime_type}")

    if result.mime_type.startswith("text/"):
        print(result.content)
    elif result.mime_type.startswith("image/"):
        image_data = base64.b64decode(result.content)

        image = plt.imread(
            io.BytesIO(image_data), format=result.mime_type.split("/")[1]
        )

        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.axis("off")

# Send Image to LLM

In [ ]:
llm = OpenAILLM()

image_as_base64 = results[-1].content
image_data = base64.b64decode(image_as_base64)
image = plt.imread(io.BytesIO(image_data), format=results[-1].mime_type.split("/")[1])
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
test_messages = [
    LLMMessage(
        content=[
            MessageContent(
                type="input_text",
                text="What is in this image?",
            ),
            MessageContent(
                type="input_image",
                image_url=image_as_base64,
            ),
        ],
        role=Roles.USER,
    ),
]

In [ ]:
answer = await llm.generate(test_messages)

print("\n\nAnswer content:")
print(answer.content)

In [ ]:
answer_stream = llm.generate_stream(test_messages)

result = ""
async for chunk in answer_stream:
    result += chunk.content[0].text if chunk.content else ""

print("\n\nAnswer content:")
print(result)

----------------------